In [2]:
import pandas as pd

# Version 1
Quick test -- output each country as its own files

In [3]:
q = '''SELECT
  date(_partitiontime) AS date,
  flag,
  integer(SUM(fishing_hours)) fishing_hours
FROM
  [world-fishing-827:gfw_draft_data.fishing_effort]
GROUP BY
  date,
  flag
ORDER BY
  flag,
  date'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False)

In [4]:
df.head()

,date,flag,fishing_hours
0,2015-08-17,MNG,24
1,2015-08-18,MNG,24
2,2015-08-19,MNG,23
3,2015-08-20,MNG,24
4,2015-08-21,MNG,22


In [5]:
len(df)

164185

In [6]:
df = df.sort_values(by=['flag', 'date']).reset_index(drop=True)
df = df.fillna(0)

In [7]:
flag = df.flag[0]
iso3s = []
header = "date,fishing\n"
rows = []
for index, row in df.iterrows():
    fl = row.flag
    if fl!=flag:
        with open("countries/{}.csv".format(flag), 'w') as f:
            f.write(header)
            f.write("\n".join(rows))
        rows = []
#         print flag
        iso3s.append(flag)
        flag = fl
    
    rows.append("{},{}".format(row.date, row.fishing_hours))
    
with open("countries/{}.csv".format(flag), 'w') as f:
    f.write(header)
    f.write("\n".join(rows))
rows = []
# print flag
iso3s.append(flag)


In [7]:
q = '''SELECT
  date(_partitiontime) AS date,
  flag,
  integer(SUM(fishing_hours)) fishing_hours
FROM
  [world-fishing-827:gfw_draft_data.fishing_effort]
  where _partitiontime >= timestamp("2016-01-01")
GROUP BY
  date,
  flag
ORDER BY
  flag,
  date'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False)

In [8]:
df = df.sort_values(by=['flag', 'date']).reset_index(drop=True).fillna(0)

In [10]:
flag = df.flag[0]
iso3s = []
header = "date,fishing\n"
rows = []
for index, row in df.iterrows():
    fl = row.flag
    if fl!=flag:
        with open("countries2016/{}.csv".format(flag), 'w') as f:
            f.write(header)
            f.write("\n".join(rows))
        rows = []
#         print flag
        iso3s.append(flag)
        flag = fl
    
    rows.append("{},{}".format(row.date, row.fishing_hours))
    
with open("countries/{}.csv".format(flag), 'w') as f:
    f.write(header)
    f.write("\n".join(rows))
rows = []
# print flag
iso3s.append(flag)

In [12]:
q = '''SELECT
  date(_partitiontime) AS date,
  integer(SUM(fishing_hours)) fishing_hours
FROM
  [world-fishing-827:gfw_draft_data.fishing_effort]
GROUP BY
  date
ORDER BY
  date'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False)

In [13]:
df = df.sort_values(by='date').reset_index(drop=True).fillna(0)

In [14]:
with open("countries/WLD.csv".format(flag), 'w') as f:
    f.write(header)
    rows = []
    for index, row in df.iterrows():
        rows.append("{},{}".format(row.date,row.fishing_hours))
    f.write("\n".join(rows))

In [ ]:
q = '''SELECT
  date(_partitiontime) AS date,
  integer(SUM(fishing_hours)) fishing_hours
FROM
  [world-fishing-827:gfw_draft_data.fishing_effort]
GROUP BY
  date
ORDER BY
  date'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False)

# Version 2 
 - put all the values in the same file
 - use local time for the days
 - include fishing vessel days

In [7]:
q = '''SELECT
  date,
  flag,
  COUNT(*) vessels_fishing,
  SUM(fishing_hours) fishing_hours
FROM (
  SELECT
    mmsi,
    DATE(SEC_TO_TIMESTAMP(TIMESTAMP_TO_SEC(timestamp) + lon/180*12*3600)) date,
    SUM(hours) fishing_hours,
  FROM
    [gfw_research.nn2]
  WHERE
    nnet_score2 = 1
  GROUP BY
    mmsi,
    date) a
LEFT JOIN
  [gfw_draft_data.fishing_vessels] b
ON
  a.mmsi = b.mmsi
  where flag not in ("CUB","MNG")
  and flag is not null
GROUP BY
  flag,
  date
ORDER BY
  flag,'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False).set_index("date")

In [5]:
len(df)

137127

In [8]:
df.to_csv("allcountries2012-2016.csv")

In [5]:
import pycountry

In [6]:
bad_countries = ["NCL", "PYF", "REU", "SGP", "SHN", "SPM", "SYC", "TCA", "TON", "TWN", "UNK", "ATF", "BES", "BHR", "COK", "CUW", "CYM", "DMA", "FLK", "FRO", "FSM", "GLP", "GRL", "GUF", "HKG", "KIR", "KNA", "MAC", "MDV", "MHL", "MLT", "MNE", "MTQ"]

In [10]:
for c in bad_countries:
    try:
        print pycountry.countries.get(alpha_3=c).name
    except:
        print c

    

New Caledonia
French Polynesia
Réunion
Singapore
Saint Helena, Ascension and Tristan da Cunha
Saint Pierre and Miquelon
Seychelles
Turks and Caicos Islands
Tonga
Taiwan, Province of China
UNK
French Southern Territories
Bonaire, Sint Eustatius and Saba
Bahrain
Cook Islands
Curaçao
Cayman Islands
Dominica
Falkland Islands (Malvinas)
Faroe Islands
Micronesia, Federated States of
Guadeloupe
Greenland
French Guiana
Hong Kong
Kiribati
Saint Kitts and Nevis
Macao
Maldives
Marshall Islands
Malta
Montenegro
Martinique


In [26]:
q = '''select flag, sum(fishing_hours)/365 hours_per_day from 
[gfw_draft_data.fishing_effort]
where flag in ({})
and flag != "UNK"
group by flag order by 2 desc'''.format('"'+'","'.join(bad_countries)+'"')

In [27]:
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827',verbose=False)

In [32]:
df['country'] = df.flag.map(lambda x: pycountry.countries.get(alpha_3=x).name)

In [33]:
df

,flag,hours_per_day,country
0,TWN,14622.066134,"Taiwan, Province of China"
1,FRO,3049.760446,Faroe Islands
2,GRL,1448.773280,Greenland
3,FLK,688.486137,Falkland Islands (Malvinas)
4,NCL,516.085764,New Caledonia
5,GUF,471.527336,French Guiana
6,REU,417.030173,Réunion
7,SYC,374.402710,Seychelles
8,FSM,310.302245,"Micronesia, Federated States of"
9,MLT,259.593892,Malta


In [38]:
to_remove = list(df.iloc[12:].flag)+[u"UNK"]
to_remove

[u'KNA',
 u'MDV',
 u'COK',
 u'MAC',
 u'SPM',
 u'HKG',
 u'SHN',
 u'BHR',
 u'PYF',
 u'DMA',
 u'CUW',
 u'ATF',
 u'BES',
 u'MTQ',
 u'MNE',
 u'TCA',
 u'TON',
 u'SGP',
 u'CYM',
 u'UNK']

In [39]:
# flag states we will keep
df[['flag','country']].iloc[:12]

,flag,country
0,TWN,"Taiwan, Province of China"
1,FRO,Faroe Islands
2,GRL,Greenland
3,FLK,Falkland Islands (Malvinas)
4,NCL,New Caledonia
5,GUF,French Guiana
6,REU,Réunion
7,SYC,Seychelles
8,FSM,"Micronesia, Federated States of"
9,MLT,Malta


In [1]:
# Append All vessels and Chinese Vessels to all allcountries2012-2016.csv

import pandas as pd

In [3]:
q = '''SELECT
  date,
  "WLD" as flag,
  COUNT(*) vessels_fishing,
  SUM(fishing_hours) fishing_hours
FROM (
  SELECT
    mmsi,
    DATE(SEC_TO_TIMESTAMP(TIMESTAMP_TO_SEC(timestamp) + lon/180*12*3600)) date,
    SUM(hours) fishing_hours,
  FROM
    [gfw_research.nn2]
  WHERE
    nnet_score2 = 1
  GROUP BY
    mmsi,
    date) a
LEFT JOIN
  [gfw_draft_data.fishing_vessels] b
ON
  a.mmsi = b.mmsi
GROUP BY
  flag,
  date
ORDER BY
  date'''
df_WLD = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False).set_index("date")

In [4]:
q = '''SELECT
  date,
  "NCH" as flag,
  COUNT(*) vessels_fishing,
  SUM(fishing_hours) fishing_hours
FROM (
  SELECT
    mmsi,
    DATE(SEC_TO_TIMESTAMP(TIMESTAMP_TO_SEC(timestamp) + lon/180*12*3600)) date,
    SUM(hours) fishing_hours,
  FROM
    [gfw_research.nn2]
  WHERE
    nnet_score2 = 1
  GROUP BY
    mmsi,
    date) a
LEFT JOIN
  [gfw_draft_data.fishing_vessels] b
ON
  a.mmsi = b.mmsi
Where
flag != "CHN"
GROUP BY
  flag,
  date
ORDER BY
  date'''
df_NCH = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False).set_index("date")

In [54]:
df_all = pd.DataFrame.from_csv("allcountries2012-2016.csv")

,flag,vessels_fishing,fishing_hours
date,,,
2011-12-31,NCH,10,78.800139
2012-01-01,NCH,39,416.334583
2012-01-02,NCH,67,672.477222
2012-01-03,NCH,102,1316.876250
2012-01-04,NCH,111,1581.026389


In [59]:
df = pd.concat([df_all, df_WLD, df_NCH])

In [60]:
df = df[~df['flag'].isin(to_remove)]


In [62]:
df.head()

,flag,vessels_fishing,fishing_hours
date,,,
2015-03-30 00:00:00,MYS,6,34.229167
2014-01-13 00:00:00,MYS,2,12.721111
2016-04-21 00:00:00,MYS,14,209.207639
2014-08-30 00:00:00,MYS,1,2.472917
2014-05-03 00:00:00,MYS,2,68.490417


In [70]:
from datetime import datetime
df =df.drop("2011-12-31")
df = df.drop("2017-01-01")
df = df.drop(datetime.strptime("2017-01-01","%Y-%m-%d"))
df = df.drop(datetime.strptime("2011-12-31","%Y-%m-%d"))

In [77]:
def stringify(d):
    try:
        return str(d.date())
    except:
        return d

df.index = df.index.map(stringify)

In [84]:
df['colFromIndex'] = df.index
df = df.sort(['flag', 'colFromIndex'])
df[['flag','vessels_fishing','fishing_hours']].to_csv('allcountries2012-2016.csv')

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  
